In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [8]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy

2018-03-16 15:24:30,914 : INFO : 'pattern' package not found; tag filters are not available for English


In [10]:
ps.stem('discus')

u'discu'

In [11]:
# Remove non-alphabets from text
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [12]:
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt',
    'LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt',
    'LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 
    'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt',
    'LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 
    'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,
    'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt',
    'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']


In [13]:
en_stop = set(get_stop_words('en'))

In [16]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))


ps = PorterStemmer()
# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [17]:
# Create all the court specific dictionaries
T = texts()
dictionary_DCDRC = corpora.Dictionary(text for text in T.DCDRC())
dictionary_NCDRC = corpora.Dictionary(text for text in T.NCDRC())
dictionary_SCDRC = corpora.Dictionary(text for text in T.SCDRC())
dictionary_DelhiHC = corpora.Dictionary(text for text in T.DelhiHC())
dictionary_JharkhandHC = corpora.Dictionary(text for text in T.JharkhandHC())
dictionary_JodhpurHC = corpora.Dictionary(text for text in T.JodhpurHC())
dictionary_KolkataHC = corpora.Dictionary(text for text in T.KolkataHC())
dictionary_SupremeCourt = corpora.Dictionary(text for text in T.SupremeCourt())
# ## check out dictionaries
# dictionary_DCDRC.save_as_text('newdictionary_DCDRC.txt') 
# dictionary_NCDRC.save_as_text('newdictionary_NCDRC.txt') 
# dictionary_SCDRC.save_as_text('newdictionary_SCDRC.txt') 
# dictionary_DelhiHC.save_as_text('newdictionary_DelhiHC.txt') 
# dictionary_JharkhandHC.save_as_text('newdictionary_JharkhandHC.txt') 
# dictionary_JodhpurHC.save_as_text('newdictionary_JodhpurHC.txt') 
# dictionary_KolkataHC.save_as_text('newdictionary_KolkataHC.txt') 
# dictionary_SupremeCourt.save_as_text('newdictionary_SupremeCourt.txt') 
# Save actual dictionaries
dictionary_DCDRC.save('1') 
dictionary_NCDRC.save('2') 
dictionary_SCDRC.save('3') 
dictionary_DelhiHC.save('4') 
dictionary_JharkhandHC.save('5') 
dictionary_JodhpurHC.save('6') 
dictionary_KolkataHC.save('7') 
dictionary_SupremeCourt.save('8')

2018-03-16 15:26:15,866 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-16 15:32:51,269 : INFO : adding document #10000 to Dictionary(172845 unique tokens: [u'gudadannavar', u'alsocont', u'beencredit', u'delhifano', u'alsocons']...)
2018-03-16 15:38:45,492 : INFO : adding document #20000 to Dictionary(312646 unique tokens: [u'gudadannavar', u'falselitig', u'againstcomplain', u'ofday', u'nonehru']...)
2018-03-16 15:44:18,352 : INFO : adding document #30000 to Dictionary(472300 unique tokens: [u'gudadannavar', u'mrmvenkatesancounsel', u'rkgejggll', u'boncer', u'metersand']...)
2018-03-16 15:49:08,979 : INFO : adding document #40000 to Dictionary(654903 unique tokens: [u'gudadannavar', u'woodb', u'mrmvenkatesancounsel', u'atsugham', u'rkgejggll']...)
2018-03-16 15:52:53,477 : INFO : adding document #50000 to Dictionary(771804 unique tokens: [u'oldwhirlpool', u'asstregistrarcoop', u'boncer', u'presidenticerakpandamembercdcas', u'khallimahapatro']...)
2018-03-16 15:56

KeyboardInterrupt: 

In [ ]:
# load those dictionaries
dictionary_DCDRC = corpora.Dictionary.load('1') 
dictionary_NCDRC = corpora.Dictionary.load('2') 
dictionary_SCDRC = corpora.Dictionary.load('3') 
dictionary_DelhiHC = corpora.Dictionary.load('4') 
dictionary_JharkhandHC = corpora.Dictionary.load('5') 
dictionary_JodhpurHC = corpora.Dictionary.load('6') 
dictionary_KolkataHC = corpora.Dictionary.load('7') 
dictionary_SupremeCourt= corpora.Dictionary.load('8')

In [ ]:
# discard words occuring in less than 5 documents and in more than 50% 
dictionary_DCDRC.filter_extremes(keep_n=None)
dictionary_NCDRC.filter_extremes(keep_n=None) 
dictionary_SCDRC .filter_extremes(keep_n=None)
dictionary_DelhiHC.filter_extremes(keep_n=None) 
dictionary_JharkhandHC.filter_extremes(keep_n=None) 
dictionary_JodhpurHC.filter_extremes(keep_n=None) 
dictionary_KolkataHC.filter_extremes(keep_n=None) 
dictionary_SupremeCourt.filter_extremes(keep_n=None) 
# Merge all the dictionaries
dictionary = dictionary_DCDRC
dictionary.merge_with(dictionary_DelhiHC)
dictionary.merge_with(dictionary_NCDRC)
dictionary.merge_with(dictionary_SCDRC)
dictionary.merge_with(dictionary_SupremeCourt)
dictionary.merge_with(dictionary_JharkhandHC)
dictionary.merge_with(dictionary_JodhpurHC)
dictionary.merge_with(dictionary_KolkataHC)
dictionary.compactify()



In [ ]:
dictionary.filter_extremes(no_below=100)
# Need this after merging
dictionary.compactify()

In [ ]:
d = copy.deepcopy(dictionary)

In [ ]:
len(dictionary)

In [ ]:
len(d)

In [ ]:
# save the dictionary
dictionary.save('newdictionary')

In [ ]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('newdictionary')

In [ ]:
len(dictionary.items())

In [ ]:
# This is BOW wrapper
c = my_corpus()
corpus = c.everything()
# save BOW
corpora.MmCorpus.serialize('newcorpus.mm', corpus)

In [ ]:
######################################## load BOW
corpus = corpora.MmCorpus('newcorpus.mm')

In [ ]:
# Tfidf model
# Will take time as to generate a model it processes over the whole corpus
tfidf = gensim.models.TfidfModel(corpus)
# save tfidf model
tfidf.save('newtfidf.mdl')

In [ ]:
##################################### load tfidf model
tfidf = gensim.models.tfidfmodel.TfidfModel.load('newtfidf.mdl')

In [ ]:
# tfidf wrapper coprus
# doesn't take time - yield
tfidf_corpus = tfidf[corpus]

# actual tfidf corpus serialized
corpora.MmCorpus.serialize('newtfidf_corpus.mm', tfidf_corpus)


In [ ]:
##################################### load tfidf corpus
tfidf_corpus = corpora.MmCorpus('newtfidf_corpus.mm')

In [ ]:
# get the query truth in dictionary
# query no. gives the list of 0-1 documents
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [ ]:
# create index - Tfidf
tfidf_index = gensim.similarities.Similarity('tfidf.index',tfidf_corpus,len(dictionary))

# save index - tfidf
gensim.similarities.Similarity.save(tfidf_index,'TFIDF_index')


In [ ]:
########################### load tfidf index
tfidf_index = gensim.similarities.Similarity.load('TFIDF_index')

In [ ]:
# calculate similarity for all 
tfidf_index.num_best= None

In [ ]:
#######################         SIMILIRATIES SCORES AND RANKING    ###########################
tfidf_results = copy.deepcopy(query_truth)
### lists of list (indexing from 1, dummy element)
sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    # inside the square bracket determines query representation
    
    sims = tfidf_index[tfidf[dictionary.doc2bow(query)]]
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in tfidf_results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        #x.append(common_words(filenames.index(x[0]+'.txt'), tfidf_corpus, query))
    
    # sort wrt relevance(from truth) and then ranks(from our model)
    #tfidf_results[str(i+1)].sort(key=lambda x: (-int(x[1]),x[3]))     
    
        

In [ ]:
tfidf_results


In [ ]:
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'tfidf.xlsx'
ws1 = wb.active
ws1.title = "TFIDF"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in tfidf_results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

In [ ]:
#evaluation_Qwise - Precision, Recall and Fscore

import pandas as pd

f = pd.ExcelFile('tfidf.xlsx')

from scipy.stats import hmean



model_name = 'tfidf'
places = 4
wb = load_workbook('tfidf.xlsx')
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = 'tfidf.xlsx')

In [ ]:
#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile('tfidf.xlsx')

from scipy.stats import hmean


model_name = 'tfidf'
places = 4
wb = load_workbook('tfidf.xlsx')
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:
    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = 'tfidf.xlsx')